In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

from py.is_rebalance import is_rebalance
from py.signals import signals
from py.split_df import split_df
from py.simulate import simulate

In [2]:
# VARIABLES

# We want 180 day windows (6 months) with 120 day overlap (4 months)
# Since our dataframe is in hours, multiply by 24
window_len = 24 * 180
overlap = 24 * 120

# Assets traded
assets = ['ETH', 'USD']

# Moving average intervals used
moving_averages = [50, 100, 200]

# Potential ETH to DAI allocations from bullish signals
bull_allocation = [
    [0.90, 0.10],
    [0.85, 0.15],
    [0.80, 0.20],
    [0.75, 0.25],
    [0.70, 0.30],
    [0.65, 0.35],
    [0.60, 0.40]
]

# List of allocations used, with the inverse allocation for bearish signals
allocation_lst = [{'bull': b,
                   'neutral': [0.50, 0.50],
                   'bear': b[::-1]}
                  for b in bull_allocation]


# Minimum difference in weighting needed to rebalance without a new signal
# This prevents unnecessary rebalancing
wiggle_room_lst = np.arange(0, 0.11, 0.01)

In [3]:
# Functions

# This loops through all backtest windows, allocations, and wiggle room and returns a dataframe with the
# resulting performance
def run(allocation_lst, wiggle_room_lst, dfs, rebalance_interval):
    
    results = []
    
    for allocation in allocation_lst:
        for wiggle_room in wiggle_room_lst:
            result = {
                'wiggle_room': wiggle_room,
                'allocation': '/'.join(str(x) for x in allocation['bull']),
            }

            # Add result for each split dataframe
            for df_split in dfs:
                start = datetime.strftime(df_split[0]['date'], '%Y.%m.%d')
                end = datetime.strftime(df_split[-1]['date'], '%Y.%m.%d')

                _, _, performance = simulate(assets, allocation, wiggle_room, df_split, rebalance_interval)

                result[start + '-' + end] = performance

            # Save result to results
            results.append(result)
            
    # Convert dict to DataFrame
    df_results = pd.DataFrame.from_records(results)
    df_results['sum'] = df_results.drop(['wiggle_room', 'allocation'], axis=1).sum(axis=1)
    
    # Sort
    df_results = df_results.sort_values('sum', ascending=False)
    
    return df_results    

In [4]:
df = pd.read_csv('../data/ETH-USDT.csv', usecols=['date', 'close']).rename({'close':'ETH'}, axis=1)
df['date'] = pd.to_datetime(df['date'])
df['USD'] = 1

# Create columns to 
df['rebalance_daily'] = is_rebalance(df['date'], day=None, hour=10)
df['rebalance_weekly'] = is_rebalance(df['date'], day='Saturday', hour=10)
df['signal'] = signals(df['ETH'], df['rebalance_daily'], *moving_averages)

# Split dataframe into windows 
dfs = split_df(df.to_dict(orient='records'), overlap, window_len)

df_results_daily = run(allocation_lst, wiggle_room_lst, dfs, 'daily')
df_results_weekly = run(allocation_lst, wiggle_room_lst, dfs, 'weekly')

# Save signals and performance to CSV
df.to_csv('backtests/signals.csv', index=False)
df_results_daily.to_csv('backtests/performance_daily.csv', index=False)
df_results_weekly.to_csv('backtests/performance_weekly.csv', index=False)

In [8]:
df_results_daily.head(10)

,wiggle_room,allocation,2017.08.16-2018.02.14,2017.10.16-2018.04.15,2017.12.15-2018.06.14,2018.02.14-2018.08.14,2018.04.15-2018.10.13,2018.06.14-2018.12.12,2018.08.14-2019.02.10,2018.10.13-2019.04.11,2018.12.12-2019.06.11,2019.02.10-2019.08.10,2019.04.11-2019.10.09,2019.06.11-2019.12.08,2019.08.09-2020.02.06,2019.10.09-2020.04.06,sum
54,0.10,0.7/0.3,-664.973095,-804.945126,361.939775,2173.990540,1100.629612,1771.516349,1449.649279,1365.842153,-853.100350,-860.215896,-351.042253,543.901146,638.989077,419.780472,6291.961682
10,0.10,0.9/0.1,-938.608943,-913.301743,473.734983,2585.938345,910.195291,1163.581866,1247.109893,2062.589662,-774.710575,-813.826535,-323.008819,520.461599,641.459903,416.217334,6257.832261
9,0.09,0.9/0.1,-939.362930,-896.871707,378.484083,2522.656697,891.757231,1135.630475,1244.539709,2057.923358,-774.710575,-799.431579,-323.008819,536.943673,641.459903,398.624214,6074.633733
2,0.02,0.9/0.1,-948.453423,-695.830105,597.278242,2542.425057,653.596874,907.356894,1050.996729,2359.107826,-700.518579,-770.705410,-303.450491,361.396377,588.251439,424.509227,6065.960656
30,0.08,0.8/0.2,-838.758229,-833.639678,427.567322,2328.090994,880.121412,1343.218069,1306.803700,1880.065557,-814.950798,-829.135078,-326.900661,520.878990,620.531578,398.054359,6061.947538
20,0.09,0.85/0.15,-888.552805,-856.300310,436.356065,2370.115621,865.427792,1230.172623,1278.539345,2005.475040,-807.238518,-822.358115,-313.736852,512.467867,615.263570,412.974646,6038.605968
1,0.01,0.9/0.1,-947.830799,-686.323946,603.148202,2554.067697,653.136040,913.851334,1031.604971,2343.069710,-709.203528,-777.005815,-311.347732,349.345086,579.254790,432.627010,6028.393018
0,0.00,0.9/0.1,-940.598566,-675.427406,594.714463,2563.237480,649.008153,910.690174,1030.851123,2342.522069,-713.249526,-775.978583,-306.104454,339.958364,568.570143,434.792743,6022.986177
5,0.05,0.9/0.1,-956.537587,-817.051253,418.480068,2457.945644,721.226165,998.112643,1175.889793,2325.436628,-665.617648,-773.065079,-297.476608,395.292974,608.575753,408.718465,5999.929958
8,0.08,0.9/0.1,-918.310378,-838.170963,431.504316,2527.871947,837.870882,1058.641489,1218.060450,2053.104501,-777.890348,-809.223607,-345.558417,497.585992,641.728970,405.889646,5983.104480


In [9]:
df_results_weekly.head(10)

,wiggle_room,allocation,2017.08.16-2018.02.14,2017.10.16-2018.04.15,2017.12.15-2018.06.14,2018.02.14-2018.08.14,2018.04.15-2018.10.13,2018.06.14-2018.12.12,2018.08.14-2019.02.10,2018.10.13-2019.04.11,2018.12.12-2019.06.11,2019.02.10-2019.08.10,2019.04.11-2019.10.09,2019.06.11-2019.12.08,2019.08.09-2020.02.06,2019.10.09-2020.04.06,sum
10,0.10,0.9/0.1,-747.325679,-332.360418,752.221696,2073.866756,1221.497841,2455.230904,1919.909730,1673.398288,-1089.125145,-825.977314,-34.806333,793.930676,548.706342,563.204556,8972.371900
9,0.09,0.9/0.1,-803.673218,-332.360418,752.221696,2054.723307,1195.138312,2393.513259,1758.102824,1618.018982,-1089.125145,-825.977314,-34.806333,793.930676,546.720891,530.020698,8556.448217
8,0.08,0.9/0.1,-803.673218,-332.360418,752.221696,2054.723307,1195.138312,2393.513259,1758.102824,1618.018982,-1098.977227,-828.282679,-44.027735,765.308983,540.511454,499.349290,8469.566830
7,0.07,0.9/0.1,-803.673218,-360.250382,678.418562,2081.527164,1212.786722,2378.054235,1730.436916,1603.664100,-1115.961217,-828.282679,-44.027735,765.308983,540.511454,499.349290,8337.862194
6,0.06,0.9/0.1,-790.201782,-333.339089,737.853552,2059.360342,1162.105398,2303.586440,1775.176552,1603.274579,-1116.107380,-831.527922,-56.203349,765.308983,540.511454,508.973436,8328.771215
19,0.08,0.85/0.15,-729.453268,-372.128274,663.238996,1982.533402,1155.055067,2335.277395,1854.711006,1551.490892,-1054.661190,-831.093177,-89.634775,765.426239,556.893984,501.050976,8288.707271
21,0.10,0.85/0.15,-736.237384,-411.724915,721.889748,1999.663720,1181.867113,2322.855536,1720.834231,1531.082805,-1013.944871,-869.830811,-121.778869,786.093514,692.442656,428.396818,8231.609293
20,0.09,0.85/0.15,-736.237384,-425.388094,539.084673,1999.663720,1182.915433,2373.511398,1854.711006,1531.082805,-1004.543371,-828.978647,-81.279580,765.525218,556.893984,433.829328,8160.790488
5,0.05,0.9/0.1,-790.201782,-333.339089,737.853552,1987.247355,1172.861380,2292.709951,1699.970273,1498.592816,-1116.107380,-832.267153,-56.203349,762.327705,536.832517,525.245470,8085.522266
18,0.07,0.85/0.15,-778.033001,-372.128274,663.238996,1982.533402,1152.518427,2326.446238,1716.635597,1551.490892,-1064.869437,-831.093177,-89.634775,765.426239,549.634659,477.999443,8050.165226


### Conclusion: Fart Set will adjust ETH allocation to 90%-10% for long/short signals.  Additional rebalances with the same allocation require a 10% weight difference to trigger.